# State estimation using Kalman filtering

This notebook will be your playground for the next few hours, where you try to develop your own Kalman filter and complete the tasks set in the script.

You should begin by importing the `numpy` and `pyplot` packages (these are needed for array operations and plotting). Give Jupyter the magic command `%matplotlib inline` in order to embed the produced plots in the notebook. 

In [ ]:
# solutions/preamble.py

Now we are ready to rock and roll!

## Task 1 - Reading and preprocessing the temperature data

The data are is stored in the files `example1.txt` and `example2.txt`. You can have a look at the contents of these files using a text processor like notepad.

We need to load the data into our notebook. There are multiple ways to perform this. Some possibilities are:
1. Use the `open()` statement and write your own line parser.
2. Use the `np.loadtxt()` function.
3. Import the data into a pandas dataframe.

In [ ]:
# solutions/pandas.py

So you've got the data in the notebook. Now you should do the time conversion from the `YYYY-MM-DD HH:MM:SS` format to total elapsed seconds since the beginning. Don't forget to convert the temperature values to K.

In [ ]:
# solutions/time_to_seconds.py

To make sure we've done things right, plot the imported temperature measurements vs time. The total batch run should be slightly less than 90 hours long. Do the temperature values look ok? You can find a notebook on plotting basics in the support content folder.

In [ ]:
# solutions/plot_raw_measurements.py

## Task 2 - Setting up the process matrices

It is now time to write the process and measurement equations in linear state-space form. 



### State-to-state matrix or process matrix

In [ ]:
# solutions/state_to_state.py

### Input-to-state matrix

For the moment let's just neglect all of the inputs. While this is physically incorrect it simplifies things and helps us focus on the Kalman filter. Simply fill the input-to-state matrix $B$ with zeros and put zeros in the input vector $u$. You can use the numpy function `np.zeros()` to do this (or just insert the values one by one).

In [ ]:
# solutions/input_to_state.py

### Measurement matrix

Assume for the moment you have a single temperature sensor $T_1$. The measurement equations is given by $$ y = Cx,$$ where $x = \begin{bmatrix}T & T_j\end{bmatrix}^T$ for our simple example. Fill in the values of $C$.

In [ ]:
# %load solutions/measurement_matrix.py

### Process and measurement noise

Something we haven't discussed yet is the role of noise in the Kalman filter. 

The measurement noise is fairly understandable. We assume at each measurement update, that our sensor was perturbed by white noise according to the sensor variance.

In [ ]:
# Process noise
evolution_noise = 0.01
Q = evolution_noise*np.eye(2)

In [ ]:
# Individual temperature sensor variances
vt1 = 1.63537548
vt2 = 0.13440874
vt3 = 0.12293816

# Measurement noise
R = np.diag([vt1,vt2,vt3])

## Task 3 - Implementing the Kalman filter

In [ ]:
# solutions/kalman_filter.py

def predict(x,u,A,B,P,Q):
    """
    Prediction step of the Kalman filter
    
    Input:
    x - state vector
    u - input vector
    A - process matrix
    B - input-to-state matrix
    P - state covariance matrix
    Q - evolution noise
    
    Returns:
    x - predicted state
    P - predicted covariance
    """
    return NotImplementedError()

In [ ]:
def update(y,xpls,C,P,R):
    """
    Update step of the Kalman filter
    
    Input:
    y - measurement
    xpls - predicted state
    C - measurement matrix
    P - covariance matrix
    R - measurement noise
    
    Returns:
    x - updated step after measurement
    P - updated covariance
    K - Kalman gain
    """
    return NotImplementedError()

## Task 4 - Filtering the measurement data

In [ ]:
# Initial state guess
x0 = np.array([15,-10])
P = A.dot(5*np.eye(2).dot(A.T)) # not exactly sure how to set this...
x = x0

# Number of measurements
imax = dataset.shape[0]

# Prepare storage for filtered values
filtered_vals = np.empty((imax,2))

filtered_vals[0,:] = x0

for i in range(imax):
    
    filtered_vals[i] = x
    
    xpls,P = predict(x=x,u=u,A=A,B=B,P=P,Q=Q)
    
    y = dataset.iloc[i,1:4].values
    
    x,P,K = update(y=y,xpls=xpls,C=C,P=P,R=R)
    
print("Filtering finished!")

Looking at the state variables they very quickly converge to the same value. This is a problem of our model, which assumes exponential decay to the same value. Essentially we are missing the source terms (coolant valve, external losses, heat coming from the aeration, and yeast metabolic heat), that add to the energy balance and make $T \neq T_j$.

In [ ]:
ax = dataset.plot(x='hours',y=[r"$T_1$", r"$T_2$", r"$T_3$"])

# add filtered stuff two plot
ax.plot(dataset['hours'],filtered_vals[:,0],label="Reactor")
ax.plot(dataset['hours'],filtered_vals[:,1],label="Jacket")
ax.legend(loc=0) # update legend

# make things look nice
ax.set_xlabel("Time (h)",fontsize=12)
ax.set_ylabel("Temperature (°C)",fontsize=12)
ax.set_title("Filter works! Kind of.",fontsize=12)

ax.set_ylim([7,11]) # only look at first few hours

plt.show()